In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')


In [ ]:
import re
from os import listdir
from os.path import isfile, join
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from random import random
import tensorflow as tf
from keras import layers
import cv2
import mediapipe as mp

In [ ]:
# Make numpy values easier to read
np.set_printoptions(precision=3, suppress=True)
root_path = '/content/drive/My Drive/Colab Notebooks/course_dataset/'
gestures= ['A','B','C','L','U','R']

In [ ]:
path = '/content/drive/My Drive/Colab Notebooks/course_dataset/ASL_letter_A/annotations.csv'
ASL_train = pd.read_csv(path,
      names=["ID","frame", "gesture", "joint", "person_idx", "video_idx", "x", "y"])

  # Drop unneeded columns
group_train = ASL_train.drop(['ID', 'gesture', 'joint', 'video_idx'], axis=1)

  # Group and re-index by person_idx and frame
grouped_out = group_train.set_index(['person_idx', 'frame', group_train.groupby(['person_idx', 'frame']).cumcount()+1]).unstack().sort_index(level=1, axis=1)

  # Remove and reset the new indexes (Which were person_idx + frame)
grouped_out.reset_index(drop=True, inplace=True)

grouped_out.columns = grouped_out.columns.to_flat_index()

grouped_out.drop(grouped_out.tail(1).index,inplace=True) # drop last n rows

# Add the gesture
grouped_out['gesture']=0

gestures= ['B','C','L','U','R']
finalDF = grouped_out

for gestureIndex, g in enumerate(gestures):
  index = gestureIndex + 1
  path = '/content/drive/My Drive/Colab Notebooks/course_dataset/ASL_letter_{}/annotations.csv'.format(g)
  ASL_train = pd.read_csv(
     path,
      names=["ID","frame", "gesture", "joint", "person_idx", "video_idx", "x", "y"])

  # Drop unneeded columns
  group_train = ASL_train.drop(['ID', 'gesture', 'joint', 'video_idx'], axis=1)

  # Group and re-index by person_idx and frame
  grouped_out = group_train.set_index(['person_idx', 'frame', group_train.groupby(['person_idx', 'frame']).cumcount()+1]).unstack().sort_index(level=1, axis=1)

  # Remove and reset the new indexes (Which were person_idx + frame)
  grouped_out.reset_index(drop=True, inplace=True)

  grouped_out.columns = grouped_out.columns.to_flat_index()

  grouped_out.drop(grouped_out.tail(1).index,inplace=True) # drop last n rows
  #print(len(grouped_out))
  # Add the gesture
  print(index)
  grouped_out['gesture']=index
  grouped_out
  finalDF = finalDF.append(grouped_out)
finalDF = finalDF.reset_index(drop=True)
finalDF = finalDF.fillna(0)

In [ ]:
print(finalDF.shape)
print(finalDF)

In [ ]:
from keras.models import Sequential
import keras.layers as layers
from keras import regularizers
from sklearn.model_selection import train_test_split

# Prepare data
ASL_features = finalDF.copy()
ASL_labels = np.array(ASL_features.pop('gesture'))

ASL_features_np = np.array(ASL_features).astype('float32')
ASL_features_np = ASL_features_np.tolist()
ASL_features_np = np.array(ASL_features_np)

# Split into test, train & validaiton
X = ASL_features_np
y = ASL_labels
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.5)
x_test, x_val, y_test, y_val = train_test_split(x_test, y_test, test_size=0.5)

model = Sequential()
model.add(layers.Dense(1024, activation = 'relu',use_bias=True))
model.add(layers.Dropout(0.6))
model.add(layers.Dense(256, activation = 'relu'))
model.add(layers.Dense(6, activation = 'softmax'))

model.compile("adam", "sparse_categorical_crossentropy", metrics=["accuracy"])

# Train the model
hist = model.fit(x=x_train, y=y_train, validation_data=(x_val, y_val), batch_size=256,  epochs=15)

plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper right')
plt.show()

plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
from keras.utils.vis_utils import plot_model

model.summary()
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)